In [3]:
import cv2
import numpy as np

# step1 設定起始&結束幀
frame_seq = 14 * 30
stop_frame_seq = 25 * 30

out_size = (480, 270)
cap = cv2.VideoCapture('WiiPlay.mp4')
if not cap.isOpened():
    raise IOError("Cannot open the video file")
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_seq)

def nothing(pos):
    pass

# step3 創建Window和trackbar
cv2.namedWindow('Input')
cv2.namedWindow('Test')
cv2.createTrackbar('HueMin', 'Test', 30, 180, nothing)
cv2.createTrackbar('HueMax', 'Test', 150, 180, nothing)
cv2.createTrackbar('SatMin', 'Test', 10, 255, nothing)
cv2.createTrackbar('SatMax', 'Test', 120, 255, nothing)
cv2.createTrackbar('ValMin', 'Test', 50, 255, nothing)
cv2.createTrackbar('ValMax', 'Test', 200, 255, nothing)


while True:
    ret, frame = cap.read()
    if not ret:
        break
    cur_frame = cv2.resize(frame, out_size, 0, 0, interpolation=cv2.INTER_AREA)

    cv2.imshow('Input', cur_frame)
    
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    hue_min = cv2.getTrackbarPos('HueMin', 'Test')
    hue_max = cv2.getTrackbarPos('HueMax', 'Test')
    sat_min = cv2.getTrackbarPos('SatMin', 'Test')
    sat_max = cv2.getTrackbarPos('SatMax', 'Test')
    val_min = cv2.getTrackbarPos('ValMin', 'Test')
    val_max = cv2.getTrackbarPos('ValMax', 'Test')

    lower_color = np.array([hue_min, sat_min, val_min])
    upper_color = np.array([hue_max, sat_max, val_max])
    
    mask = cv2.inRange(hsv, lower_color, upper_color)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    mask = cv2.erode(mask, kernel)
    mask = cv2.dilate(mask, kernel)
    cv2.imshow('Test', mask)
    
    
    lower_blue = np.array([100, 150, 50])
    upper_blue = np.array([125, 255, 255])
    blue_mask = cv2.inRange(hsv, lower_blue, upper_blue)
    cv2.imshow('Cursor', blue_mask)
    
    lower_yellow = np.array([15, 200, 160])
    upper_yellow = np.array([30, 255, 255])
    yellow_mask = cv2.inRange(hsv, lower_yellow, upper_yellow)
    cv2.imshow('Timer', yellow_mask)
    
    
    lower_skin = np.array([5, 25, 50])
    upper_skin = np.array([30, 255, 255])
    
    skin_mask = cv2.inRange(hsv, lower_skin, upper_skin)
    skin_mask = cv2.erode(skin_mask, kernel)
    skin_mask = cv2.dilate(skin_mask, kernel)    
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (8, 8))
    skin_mask = cv2.dilate(skin_mask, kernel)
    cv2.imshow('Skin', skin_mask)

    if cap.get(cv2.CAP_PROP_POS_FRAMES) == stop_frame_seq:
        break

    key = cv2.waitKey(1)
    if key == 32:
        cv2.waitKey()
    elif key == 27:
        break

cap.release()
cv2.destroyAllWindows()
